In [1]:
%%capture
import sys
!{sys.executable} -m pip install SimpleITK==2.2.1 cassandra-driver==3.27.0 diskcache==4.1.0 \
    ipympl==0.9.3 ipyvolume==0.6.1

In [2]:
import ipyvolume as ipv
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage.morphology
from p2ch10.dsets import getCandidateInfoList, getCt, LunaDataset
from p2ch10.vis import findPositiveSamples, showCandidate

In [ ]:
candidateInfo_list = getCandidateInfoList(requireOnDisk_bool=True)
positiveInfo_list = [x for x in candidateInfo_list if x[0]]
diameter_list = [x[1] for x in positiveInfo_list]

In [ ]:
print(len(positiveInfo_list))
print(positiveInfo_list[0])

In [ ]:
for i in range(0, len(diameter_list), 100):
    print('{:4}  {:4.1f} mm'.format(i, diameter_list[i]))

In [ ]:
for candidateInfo_tup in positiveInfo_list[:3]:
    print(candidateInfo_tup)
for candidateInfo_tup in positiveInfo_list[-3:]:
    print(candidateInfo_tup)
    
for candidateInfo_tup in positiveInfo_list:
    if candidateInfo_tup.series_uid.endswith('565'):
        print(candidateInfo_tup)

In [ ]:
np.histogram(diameter_list)

In [ ]:
%matplotlib inline
_ = plt.hist(diameter_list, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram of nodule diameters")

In [ ]:
positiveSample_list = findPositiveSamples(start_ndx=0, limit=11)

In [ ]:
series_uid = positiveSample_list[0][2]
series_uid

In [ ]:
candidateInfo_list[0]

In [ ]:
# %matplotlib inline
series_uid = '1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565'
showCandidate(series_uid)

In [ ]:
# %matplotlib inline
series_uid = '1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886'
showCandidate(series_uid)

In [ ]:
LunaDataset()[0]

In [ ]:
# V = np.zeros((128,128,128)) # our 3d array
# # outer box
# V[30:-30,30:-30,30:-30] = 0.75
# V[35:-35,35:-35,35:-35] = 0.0
# # inner box
# V[50:-50,50:-50,50:-50] = 0.25
# V[55:-55,55:-55,55:-55] = 0.0
# ipv.quickvolshow(V, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)


In [ ]:
# ct = getCt(series_uid)
# ipv.quickvolshow(ct.hu_a, level=[0.25, 0.5, 0.9], opacity=0.1, level_width=0.1, data_min=-1000, data_max=1000)

# Note: the following visualization doesn't look very good.
It's only included here for completeness. 

In [ ]:
def build2dLungMask(ct, mask_ndx, threshold_gcc = 0.7):
    dense_mask = ct.hu_a[mask_ndx] > threshold_gcc
    denoise_mask = scipy.ndimage.morphology.binary_closing(dense_mask, iterations=2)
    tissue_mask = scipy.ndimage.morphology.binary_opening(denoise_mask, iterations=10)
    body_mask = scipy.ndimage.morphology.binary_fill_holes(tissue_mask)
    air_mask = scipy.ndimage.morphology.binary_fill_holes(body_mask & ~tissue_mask)

    lung_mask = scipy.ndimage.morphology.binary_dilation(air_mask, iterations=2)

    return air_mask, lung_mask, dense_mask, denoise_mask, tissue_mask, body_mask


def build3dLungMask(ct):
    air_mask, lung_mask, dense_mask, denoise_mask, tissue_mask, body_mask = mask_list = \
        [np.zeros_like(ct.hu_a, dtype=np.bool) for _ in range(6)]

    for mask_ndx in range(ct.hu_a.shape[0]):
        for i, mask_ary in enumerate(build2dLungMask(ct, mask_ndx)):
            mask_list[i][mask_ndx] = mask_ary

    return air_mask, lung_mask, dense_mask, denoise_mask, tissue_mask, body_mask

In [ ]:
ct = getCt(series_uid)
air_mask, lung_mask, dense_mask, denoise_mask, tissue_mask, body_mask = build3dLungMask(ct)


In [ ]:
# bones = ct.hu_a * (ct.hu_a > 1.5)
# lungs = ct.hu_a * air_mask
# ipv.figure()
# ipv.pylab.volshow(bones + lungs, level=[0.17, 0.17, 0.23], data_min=100, data_max=900)
# ipv.show()